In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from loguru import logger

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

# Scrape lots and expiries from Zerodha

In [2]:
import requests
import json

from bs4 import BeautifulSoup
import re

In [3]:
data = {"action": "calculate",
        "exchange%5B%5D": "NFO",
        "product%5B%5D": "OPT",
        "scrip%5B%5D": "AARTIIND24JUL",
        "option_type%5B%5D": "PE",
        "strike_price%5B%5D": 660,
        "qty%5B%5D": 1000,
        "trade%5B%5D": "sell"}

In [4]:
def get_zerodha_span_payload(data: dict):

    zero_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                    'AppleWebKit/537.36 '
                                    '(KHTML, like Gecko) '
                                    'Chrome/80.0.3987.149 Safari/537.36',
                    'referer': 'https://zerodha.com/',
                    'origin': 'https://zerodha.com/',
                    'content-type': 'application/json',
                    }
    
    base_url = "https://zerodha.com/"
    url = "https://zerodha.com/margin-calculator/SPAN"


    with requests.Session() as session:
        request = session.get(base_url, headers=zero_headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.post(url, headers=zero_headers, data=data, timeout=5, cookies=cookies)

    return response

In [42]:
def zerodha_lots_expiries() -> pd.DataFrame:

    """Gets the lots and expiries for Zerodha"""
    
    response = get_zerodha_span_payload({}) # Empty data dictionary
    r = response.text
    soup = BeautifulSoup(r)
    # get all the scripts
    js_code = soup.findAll("script")
    # get the script with FUTURES variable
    the_script = [j for j in js_code if 'var FUTURES' in str(j)]

    lots_n_expiries = eval(re.findall(r'\[(.*?)\]\]', str(the_script))[0]+']')
    lot_exp_list = list([L0, L1.split()[0], L1.split()[1], L2] for L0, L1, L2 in lots_n_expiries)

    df = pd.DataFrame(lot_exp_list, columns=['zeroscrip', 'symbol', 'zexpiry', 'lot'])

    return df

In [44]:
df = zerodha_lots_expiries()

In [45]:
df[df.symbol == 'NIFTY']

,zeroscrip,symbol,zexpiry,lot
403,NIFTY24JUL,NIFTY,25-JUL-2024,25
404,NIFTY24JUN,NIFTY,27-JUN-2024,25
405,NIFTY24MAY,NIFTY,30-MAY-2024,25
577,NIFTY24613,NIFTY,13-JUN-2024,25
579,NIFTY24620,NIFTY,20-JUN-2024,25
580,NIFTY24523,NIFTY,23-MAY-2024,25
584,NIFTY2466,NIFTY,06-JUN-2024,25
